# 08-02: TypeBox 和 Ajv 数据验证TypeBox 提供 JSON Schema 生成，Ajv 提供高性能验证。

In [ ]:
// 安装: npm install @sinclair/typebox ajv
import { Type, Static } from '@sinclair/typebox';
import Ajv from 'ajv';
// ========== 1. TypeBox 基础 ==========
const UserSchema = Type.Object({
  id: Type.Integer(),
  name: Type.String({ minLength: 1, maxLength: 100 }),
  email: Type.String({ format: 'email' }),
  age: Type.Optional(Type.Integer({ minimum: 0, maximum: 150 })),
  isActive: Type.Boolean({ default: true }),
  tags: Type.Array(Type.String()),
  metadata: Type.Optional(Type.Record(Type.String(), Type.Unknown()))
});
// 提取 TypeScript 类型
type User = Static<typeof UserSchema>;
// {
//   id: number;
//   name: string;
//   email: string;
//   age?: number;
//   isActive: boolean;
//   tags: string[];
//   metadata?: Record<string, unknown>;
// }
console.log('Schema:', JSON.stringify(UserSchema, null, 2));

In [ ]:
// ========== 2. 使用 Ajv 验证 ==========
const ajv = new Ajv({
  allErrors: true,
  strict: true,
  formats: {
    email: /^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$/
  }
});
// 编译 Schema
const validate = ajv.compile(UserSchema);
// 验证数据
const data = {
  id: 1,
  name: 'John Doe',
  email: 'john@example.com',
  age: 30,
  isActive: true,
  tags: ['developer', 'admin']
};
const valid = validate(data);
console.log('Valid:', valid);
if (!valid) {
  console.log('Errors:', validate.errors);
}

In [ ]:
// ========== 3. 验证错误处理 ==========
function formatErrors(errors: typeof validate.errors): string {
  if (!errors) return '';
  
  return errors.map(err => {
    const path = err.instancePath || 'root';
    return `${path}: ${err.message}`;
  }).join('\n');
}
const badData = {
  id: 'not-a-number',
  name: '',
  email: 'invalid-email',
  age: -5,
  isActive: 'yes',
  tags: 'not-an-array'
};
const isValid = validate(badData);
if (!isValid) {
  console.log('Validation errors:');
  console.log(formatErrors(validate.errors));
}

In [ ]:
// ========== 4. 高级 Schema 类型 ==========
// Union 类型
const StatusSchema = Type.Union([
  Type.Literal('pending'),
  Type.Literal('active'),
  Type.Literal('inactive')
]);
// Enum 类型
const RoleSchema = Type.Enum({
  Admin: 'admin',
  User: 'user',
  Guest: 'guest'
});
// 交叉类型
const TimestampsSchema = Type.Object({
  createdAt: Type.String({ format: 'date-time' }),
  updatedAt: Type.String({ format: 'date-time' })
});
const FullUserSchema = Type.Intersect([UserSchema, TimestampsSchema]);
type FullUser = Static<typeof FullUserSchema>;
// 递归类型
const CategorySchema = Type.Recursive(This => Type.Object({
  id: Type.Integer(),
  name: Type.String(),
  children: Type.Array(This)
}));
type Category = Static<typeof CategorySchema>;

In [ ]:
// ========== 5. 自定义校验 ==========
// 添加自定义 format
ajv.addFormat('phone', {
  type: 'string',
  validate: (phone: string) => {
    return /^\+?[1-9]\d{1,14}$/.test(phone);
  }
});
// 添加自定义 keyword
ajv.addKeyword({
  keyword: 'range',
  type: 'number',
  compile: (schema: [number, number]) => {
    return (data: number) => data >= schema[0] && data <= schema[1];
  },
  errors: true
});
const CustomSchema = Type.Object({
  phone: Type.String({ format: 'phone' }),
  score: Type.Number({ range: [0, 100] })
});
const customValidate = ajv.compile(CustomSchema);
const result = customValidate({ phone: '+1234567890', score: 50 });
console.log('Custom validation:', result);

In [ ]:
// ========== 6. 与 Express/Fastify 集成 ==========
// Express 中间件示例
import { Request, Response, NextFunction } from 'express';
import { TSchema } from '@sinclair/typebox';
function validateBody<T extends TSchema>(schema: T) {
  const validate = ajv.compile(schema);
  
  return (req: Request, res: Response, next: NextFunction) => {
    const valid = validate(req.body);
    if (!valid) {
      return res.status(400).json({
        error: 'Validation failed',
        details: validate.errors
      });
    }
    next();
  };
}
// 使用
// app.post('/users', validateBody(UserSchema), (req, res) => {
//   // req.body 已被验证为 User 类型
// });
// Fastify 原生支持 TypeBox
// import Fastify from 'fastify';
// const app = Fastify();
// app.post('/users', { schema: { body: UserSchema } }, handler);